In [1]:
from util import *
from glob import glob
import matplotlib.pyplot as plt
from shapely import wkt
pd.set_option("display.max_columns", None)

In [6]:
gdf = gpd.read_file(f"DSAS_TestData/transects_sample_intersects.geojson")
gdf = enrich_df(gdf)
gdf

AttributeError: 'GeoDataFrame' object has no attribute 'ShorelineI'

In [ ]:
transect_metadata = get_transect_metadata(f"DSAS_TestData/transects_sample_transects.geojson")